In [2]:
import numpy as np, random, operator, pandas as pd

import networkx as nx

In [3]:
class City:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def distance(self, city):
        xDis = abs(self.x - city.x)
        yDis = abs(self.y - city.y)
        distance = np.sqrt((xDis ** 2) + (yDis ** 2))
        return distance

    def __repr__(self):
        return "(" + str(self.x) + "," + str(self.y) + ")"


In [4]:
class Fitness:
    def __init__(self, route):
        self.route = route
        self.distance = 0
        self.fitness= 0.0

    def routeDistance(self):
        if self.distance ==0:
            pathDistance = 0
            for i in range(0, len(self.route)):
                fromCity = self.route[i]
                toCity = None
                if i + 1 < len(self.route):
                    toCity = self.route[i + 1]
                else:
                    toCity = self.route[0]
                pathDistance += fromCity.distance(toCity)
            self.distance = pathDistance
        return self.distance

    def routeFitness(self):
        if self.fitness == 0:
            self.fitness = 1 / float(self.routeDistance())
        return self.fitness


In [5]:
def createRoute(cityList):
    route = random.sample(cityList, len(cityList))
    return route

In [6]:
def initialPopulation(popSize, cityList):
    population = []

    for i in range(0, popSize):
        population.append(createRoute(cityList))
    return population

In [7]:
def rankRoutes(population):
    fitnessResults = {}
    for i in range(0,len(population)):
        fitnessResults[i] = Fitness(population[i]).routeFitness()
    return sorted(fitnessResults.items(), key = operator.itemgetter(1), reverse = True)


In [8]:
def selection(popRanked, eliteSize):
    selectionResults = []
    df = pd.DataFrame(np.array(popRanked), columns=["Index","Fitness"])
    df['cum_sum'] = df.Fitness.cumsum()
    df['cum_perc'] = 100*df.cum_sum/df.Fitness.sum()

    for i in range(0, eliteSize):
        selectionResults.append(popRanked[i][0])
    for i in range(0, len(popRanked) - eliteSize):
        pick = 100*random.random()
        for i in range(0, len(popRanked)):
            if pick <= df.iat[i,3]:
                selectionResults.append(popRanked[i][0])
                break
    return selectionResults

In [9]:
def matingPool(population, selectionResults):
    matingpool = []
    for i in range(0, len(selectionResults)):
        index = selectionResults[i]
        matingpool.append(population[index])
    return matingpool

In [10]:
def breed(parent1, parent2):
    child = []
    childP1 = []
    childP2 = []

    geneA = int(random.random() * len(parent1))
    geneB = int(random.random() * len(parent1))

    startGene = min(geneA, geneB)
    endGene = max(geneA, geneB)

    for i in range(startGene, endGene):
        childP1.append(parent1[i])

    childP2 = [item for item in parent2 if item not in childP1]

    child = childP1 + childP2
    return child


In [11]:
def breedPopulation(matingpool, eliteSize):
    children = []
    length = len(matingpool) - eliteSize
    pool = random.sample(matingpool, len(matingpool))

    for i in range(0,eliteSize):
        children.append(matingpool[i])

    for i in range(0, length):
        child = breed(pool[i], pool[len(matingpool)-i-1])
        children.append(child)
    return children

In [12]:
def mutate(individual, mutationRate):
    for swapped in range(len(individual)):
        if(random.random() < mutationRate):
            swapWith = int(random.random() * len(individual))

            city1 = individual[swapped]
            city2 = individual[swapWith]

            individual[swapped] = city2
            individual[swapWith] = city1
    return individual

In [13]:
def mutatePopulation(population, mutationRate):
    mutatedPop = []

    for ind in range(0, len(population)):
        mutatedInd = mutate(population[ind], mutationRate)
        mutatedPop.append(mutatedInd)
    return mutatedPop

In [14]:
def nextGeneration(currentGen, eliteSize, mutationRate):
    popRanked = rankRoutes(currentGen)
    selectionResults = selection(popRanked, eliteSize)
    matingpool = matingPool(currentGen, selectionResults)
    children = breedPopulation(matingpool, eliteSize)
    nextGeneration = mutatePopulation(children, mutationRate)
    return nextGeneration


In [15]:
def geneticAlgorithmPlot(population, popSize, eliteSize, mutationRate, generations,rewiring):
    pop = initialPopulation(popSize, population)
    #initial watts and strogatz network
    G=nx.generators.erdos_renyi_graph(popSize,0.5)

    scores = []

    for l in range(0,generations):
        gen_scores=[]
        for j in range(0, popSize):
            candidato = pop[j]
            vecinos=[]
            for k in G[j]:
                vecinos.append(k)

            if(len(vecinos)>0):
                #Select best neighbour
                mejor_vecino = 0
                puntaje_mejor_vecino = 100000000

                for k in G[j]:
                    fitness_vecino = 1/Fitness(pop[k]).routeFitness()
                    if(fitness_vecino<puntaje_mejor_vecino):
                        puntaje_mejor_vecino = fitness_vecino
                        mejor_vecino = k

                if(puntaje_mejor_vecino<100000000):
                    pareja = pop[mejor_vecino]
                    child = breed(candidato, pareja)
                    child = mutate(child,mutationRate)

                    score_individuo  = 1/Fitness(candidato).routeFitness()
                    score_child = 1/Fitness(child).routeFitness()

                    if(score_child < score_individuo):
                        pop[j] =  child
                        G.add_edge(j,mejor_vecino)
                        gen_scores.append(score_child)
                    else:
                        gen_scores.append(score_individuo)

        for j in range(0, popSize):
            aristas = G.adj[j]

            vecinos=[]
            for k in aristas:
                vecinos.append(k)

            for i in range(0,len(vecinos)):
                if random.random() < rewiring:
                    random_index = int(random.random()*100)
                    G.remove_edge(j, vecinos[i])
                    G.add_edge(j,random_index)

        gen_best = min(gen_scores)
        scores.append(gen_best)

    return scores

In [16]:
%%time
cityList = []

#25 cities
for i in range(0,5):
    cityList.append(City(x=int(random.random() * 200), y=int(random.random() * 200)))

iteraciones = []
promedios = []
#Number of iterations per run
maxiter = 100
#Number of experiments, to then gained the average from them in each iteration
numero_iteraciones = 100

for i in range(0,numero_iteraciones):
    print(i)
    #popsize 100, elite search 20%, rewiring 30%, mutation rate 10%
    iteracionTemporal = geneticAlgorithmPlot(population=cityList, popSize=100, eliteSize=20, mutationRate=0.01, generations=maxiter,rewiring = 0.3)
    iteraciones.append(iteracionTemporal)

for i in range(0,maxiter):
    sumatoria_temporal = 0
    for j in range(0,numero_iteraciones):
        sumatoria_temporal = sumatoria_temporal + iteraciones[j][i]

    promedio_temporal = sumatoria_temporal / numero_iteraciones
    promedios.append(promedio_temporal)

print("\n")
print("Print averages per iteration:")
for i in range(0,maxiter):
    print("%.9f"%promedios[i])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


Print averages per iteration:
393.601877981
393.601877981
393.601877981
393.601877981
393.601877981
393.601877981
393.601877981
393.601877981
393.601877981
393.601877981
393.601877981
393.601877981
393.601877981
393.601877981
393.601877981
393.601877981
393.601877981
393.601877981
393.601877981
393.601877981
393.601877981
393.601877981
393.601877981
393.601877981
393.601877981
393.601877981
393.601877981
393.601877981
393.601877981
393.601877981
393.601877981
393.601877981
393.601877981
393.601877981
393.601877981
393.601877981
393.601877981
393.601877981
393.601877981
393.601877981
393.601877981
393.601877981
393.601877981
393.601877981
393.601877981
393.601877981
393.601877981
393.601877981
393.60

In [17]:
%%time
cityList = []

#25 cities
for i in range(0,50):
    cityList.append(City(x=int(random.random() * 200), y=int(random.random() * 200)))

iteraciones = []
promedios = []
#Number of iterations per run
maxiter = 100
#Number of experiments, to then gained the average from them in each iteration
numero_iteraciones = 100

for i in range(0,numero_iteraciones):
    print(i)
    #popsize 100, elite search 20%, rewiring 30%, mutation rate 10%
    iteracionTemporal = geneticAlgorithmPlot(population=cityList, popSize=100, eliteSize=20, mutationRate=0.01, generations=maxiter,rewiring = 0.3)
    iteraciones.append(iteracionTemporal)

for i in range(0,maxiter):
    sumatoria_temporal = 0
    for j in range(0,numero_iteraciones):
        sumatoria_temporal = sumatoria_temporal + iteraciones[j][i]

    promedio_temporal = sumatoria_temporal / numero_iteraciones
    promedios.append(promedio_temporal)

print("\n")
print("Print averages per iteration:")
for i in range(0,maxiter):
    print("%.9f"%promedios[i])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


Print averages per iteration:
4309.983491354
4028.196394671
3847.557230337
3672.138305146
3530.459708270
3429.786826603
3338.787523566
3270.198805981
3185.182041698
3124.678695625
3056.292570043
3011.686943866
2956.884603710
2908.127258468
2871.007090100
2834.482994228
2790.007198522
2758.684471870
2731.413560648
2702.146909389
2675.335664317
2655.556327259
2631.845691512
2603.377406685
2582.897215792
2557.093934560
2536.621136208
2518.683800319
2500.054125115
2486.527476252
2468.824324222
2455.051491785
2440.257240592
2429.742842611
2415.030647085
2402.086304166
2392.582335976
2379.837800030
2367.421923372
2356.681892175
2343.891589965
2329.704597671
2318.700217024
2308.160012318
2300.657871316
229

In [18]:
%%time
cityList = []

#25 cities
for i in range(0,500):
    cityList.append(City(x=int(random.random() * 200), y=int(random.random() * 200)))

iteraciones = []
promedios = []
#Number of iterations per run
maxiter = 100
#Number of experiments, to then gained the average from them in each iteration
numero_iteraciones = 100

for i in range(0,numero_iteraciones):
    print(i)
    #popsize 100, elite search 20%, rewiring 30%, mutation rate 10%
    iteracionTemporal = geneticAlgorithmPlot(population=cityList, popSize=100, eliteSize=20, mutationRate=0.01, generations=maxiter,rewiring = 0.3)
    iteraciones.append(iteracionTemporal)

for i in range(0,maxiter):
    sumatoria_temporal = 0
    for j in range(0,numero_iteraciones):
        sumatoria_temporal = sumatoria_temporal + iteraciones[j][i]

    promedio_temporal = sumatoria_temporal / numero_iteraciones
    promedios.append(promedio_temporal)

print("\n")
print("Print averages per iteration:")
for i in range(0,maxiter):
    print("%.9f"%promedios[i])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


Print averages per iteration:
47850.836971675
46855.716124331
46164.253924241
45588.627819124
45115.696384141
44751.696355595
44414.190402820
44087.185051780
43809.122422177
43557.541996675
43337.394669184
43129.685530049
42931.721039095
42768.104019768
42566.292079209
42431.935670365
42297.000574961
42149.915883636
42041.418959852
41918.846233695
41791.989701508
41710.359028254
41610.535689937
41501.362630874
41410.960454375
41325.396646766
41241.223969766
41159.639960683
41105.466005604
41042.567778308
40956.139360922
40895.845747321
40831.417454273
40742.122681675
40699.711188547
40617.651271215
40565.419265751
40501.165433898
40445.011886865
40376.260460139
40327.464455843
40286.211886789
40229.

In [16]:
%%time
cityList = []

#25 cities
for i in range(0,25):
    cityList.append(City(x=int(random.random() * 200), y=int(random.random() * 200)))

iteraciones = []
promedios = []
#Number of iterations per run
maxiter = 100
#Number of experiments, to then gained the average from them in each iteration
numero_iteraciones = 100

for i in range(0,numero_iteraciones):
    print(i)
    #popsize 100, elite search 20%, rewiring 30%, mutation rate 10%
    iteracionTemporal = geneticAlgorithmPlot(population=cityList, popSize=100, eliteSize=20, mutationRate=0.01, generations=maxiter,rewiring = 0.3)
    iteraciones.append(iteracionTemporal)

for i in range(0,maxiter):
    sumatoria_temporal = 0
    for j in range(0,numero_iteraciones):
        sumatoria_temporal = sumatoria_temporal + iteraciones[j][i]

    promedio_temporal = sumatoria_temporal / numero_iteraciones
    promedios.append(promedio_temporal)

print("\n")
print("Print averages per iteration:")
for i in range(0,maxiter):
    print("%.9f"%promedios[i])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


Print averages per iteration:
1876.717962813
1732.355732017
1624.680167824
1545.627869749
1494.071127519
1454.328637995
1413.012413280
1384.222000617
1357.250202954
1332.595300674
1307.453290889
1288.119253405
1273.781698209
1256.336583629
1241.825768353
1230.845179275
1221.353995372
1209.946352335
1201.682658129
1192.005651175
1182.051189390
1170.444064237
1161.899023115
1155.248098932
1150.340104162
1144.725329310
1138.623252410
1132.751490265
1125.649671356
1118.381945324
1114.919930006
1108.965873872
1106.360415787
1100.646994361
1091.356176611
1087.267635509
1083.888758752
1081.729243826
1078.622120833
1074.991713087
1072.307138094
1068.517303315
1065.559230404
1061.945967476
1059.600631795
105

In [17]:
%%time
cityList = []

#25 cities
for i in range(0,1000):
    cityList.append(City(x=int(random.random() * 200), y=int(random.random() * 200)))

iteraciones = []
promedios = []
#Number of iterations per run
maxiter = 100
#Number of experiments, to then gained the average from them in each iteration
numero_iteraciones = 100

for i in range(0,numero_iteraciones):
    print(i)
    #popsize 100, elite search 20%, rewiring 30%, mutation rate 10%
    iteracionTemporal = geneticAlgorithmPlot(population=cityList, popSize=100, eliteSize=20, mutationRate=0.01, generations=maxiter,rewiring = 0.3)
    iteraciones.append(iteracionTemporal)

for i in range(0,maxiter):
    sumatoria_temporal = 0
    for j in range(0,numero_iteraciones):
        sumatoria_temporal = sumatoria_temporal + iteraciones[j][i]

    promedio_temporal = sumatoria_temporal / numero_iteraciones
    promedios.append(promedio_temporal)

print("\n")
print("Print averages per iteration:")
for i in range(0,maxiter):
    print("%.9f"%promedios[i])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


Print averages per iteration:
98690.415301995
97284.654372744
96280.210675281
95553.430614563
94888.536127328
94326.217016009
93801.943203358
93304.724226841
92917.253292583
92561.043522680
92282.107033191
91985.557812805
91717.440439475
91434.705884226
91225.990273732
91023.054137632
90794.947874126
90629.021398732
90402.851575897
90262.747200955
90068.829207019
89871.736335512
89714.516822576
89605.560670942
89480.826875259
89344.270448167
89217.509583839
89129.116327200
89018.249042755
88913.597442003
88798.372688816
88679.311624722
88577.306196937
88494.586249812
88400.705322307
88316.723993477
88221.409337447
88151.890559047
88083.657870195
87983.569863163
87955.974489000
87869.403940902
87806.